In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00


#Loading the dataset

In [ ]:
from datasets import load_dataset
df=load_dataset("Aarif1430/english-to-hindi")

README.md:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

(…)-00000-of-00001-71c2cec7402cd444.parquet:   0%|          | 0.00/21.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127705 [00:00<?, ? examples/s]

In [ ]:
df['train'][0]

{'english_sentence': "However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'hindi_sentence': 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।'}

#Creating a dataframe

In [ ]:
import pandas as pd
da = pd.DataFrame(df['train'])
da.rename(columns={'english_sentence': 'english', 'hindi_sentence': 'hindi'}, inplace=True)
da.head()

,english,hindi
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,The value of insects in the biosphere is enorm...,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ..."
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व..."


# Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings

from numpy import array, argmax, random, take
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers

warnings.filterwarnings("ignore")


# Text preprocessing

In [ ]:
# Removing punctuations and converting text to lowercase for both languages
da['english'] = da['english'].str.replace('[{}]'.format(string.punctuation), '', regex=True).str.lower()
da['hindi'] = da['hindi'].str.replace('[{}]'.format(string.punctuation), '', regex=True).str.lower()
eng_empty_indices = da[da['english'].str.strip().astype(bool) == False].index
hin_empty_indices = da[da['hindi'].str.strip().astype(bool) == False].index
remove_indices = list(set(eng_empty_indices) | set(hin_empty_indices))
da.drop(remove_indices, inplace=True)
da.reset_index(drop=True, inplace=True)


# Tokenization

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(da['english'])


sequences_eng = tokenizer_eng.texts_to_sequences(da['english'])


tokenizer_hin = Tokenizer()
tokenizer_hin.fit_on_texts(da['hindi'])


sequences_hin = tokenizer_hin.texts_to_sequences(da['hindi'])


max_length = 100
sequences_eng = pad_sequences(sequences_eng, maxlen=max_length, padding='post')
sequences_hin = pad_sequences(sequences_hin, maxlen=max_length, padding='post')

vocab_size_eng = len(tokenizer_eng.word_index) + 1
vocab_size_hin = len(tokenizer_hin.word_index) + 1

print("Vocabulary size for English subtitles:", vocab_size_eng)
print("Vocabulary size for Hindi subtitles:", vocab_size_hin)


Vocabulary size for English subtitles: 76855
Vocabulary size for Hindi subtitles: 85607


# Train-Test-Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, xtest, y_train, ytest = train_test_split(sequences_eng[:50000], sequences_hin[:50000], test_size=0.2, random_state=42)
print("Shape of y_train:", y_train.shape)


Shape of y_train: (40000, 100)


In [ ]:
eng_length = sequences_eng.shape[1]
eng_length

100

# Building model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
model = Sequential()
model.add(Embedding(input_dim=vocab_size_eng, output_dim=128,input_shape=(eng_length,), mask_zero=True))
model.add(LSTM(units=512))
model.add(RepeatVector(n=eng_length))
model.add(LSTM(units=512, return_sequences=True))
model.add(Dense(units=vocab_size_hin, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       9,837,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 512)                 │       1,312,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 100, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100, 512)            │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100, 85607)          │      43,916,391 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57,165,799 (218.07 MB)

 Trainable params: 57,165,799 (218.07 MB)

 Non-trainable params: 0 (0.00 B)

# Compiling and Fitting the model

In [ ]:
from tensorflow.keras.optimizers import Adam

# Define optimizer
adam = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=1,
                    batch_size=32)



1000/1000 ━━━━━━━━━━━━━━━━━━━━ 748s 742ms/step - accuracy: 0.8262 - loss: 2.0863 - val_accuracy: 0.8362 - val_loss: 1.4109
